# Import Libraries

In [1]:
import pandas as pd
from itertools import zip_longest
from bs4 import BeautifulSoup
from selenium import webdriver

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

C:\Users\Gilbert KH\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# code to force show all column
pd.set_option('display.max_columns', None)

# max column width
pd.set_option('display.max_colwidth', 100)

# Load Dataset

In [3]:
df = pd.read_csv('./datasets/rumah123_hyperlink.csv')

# Scrap Web Per Pages

In [4]:
#init webdriver & test BeautifulSoup
driver = webdriver.Chrome()

url="https://www.rumah123.com/jual/rumah/?q=Balikpapan"
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") #Menentukan parser HTML yang ingin kita gunakan.
print(soup.prettify()[:700]) # memberikan representasi visual dari pohon parse yang dibuat dari konten HTML mentah.

<html lang="id-ID">
 <head>
  <script async="" id="analytics-js" src="https://cdn.segment.com/analytics.js/v1/fpueXeNRJ8uBgCLuBDRDkZFAMBT0yLLb/analytics.min.js" type="text/javascript">
  </script>
  <script>
   partytown = {
                "debug": false,
                "lib": "/asset-core/script/partytown/",
                "logCalls": true,
                "logGetters": true,
                "logImageRequests": true,
                "logMainAccess": true,
                "logScriptExecution": true,
                "logSendBeaconRequests": true,
                "logStackTraces": true,
                "forward": ['dataLayer.push', 'fbq', '_hsq.push', 'ttq.track', 'ttq.page', 'ttq.load'],
 


In [5]:
# prepare variable to save the datas
id = []
judul = []
harga = []
lokasi = []
kamar_tidur = []
kamar_mandi = []
luas_tanah = []
luas_bangunan = []
carport = []
tipe_properti = []
sertifikat = []
daya_listrik = []

kamar_pembantu = []
dapur = []
ruang_makan = []
ruang_tamu = []
kondisi_perabotan = []

# Exterior
jumlah_lantai = []
pemandangan = []

# tentang properti
terjangkau_internet = []
lebar_jalan = []
sumber_air = []
hook = []
kondisi_properti = []
tipe_iklan = []
id_iklan = []

### Changed Logic
1. First initial labels of all possible data
2. Create a dictionary based on the labels
3. Create a Loop to scrap the data and store it in the dictionary
    - First, fill the dictionary with NaN for all key so that when we didn't found the data, it still in the same shape
    - Loop 1: To get the labels1 data (Kamar Tidur, Kamar Mandi, etc)
    - Loop 2: To get the labels2 data (Luas Bangunan, Luas Tanah, etc)

In [6]:
# clean dictionary
data1 = {}
data2 = {}
harga = []
judul = []
lokasi = []
harga = [np.nan]*1467
judul = [np.nan]*1467
lokasi = [np.nan]*1467

# define the labels
labels1 = ['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Carport', 'Tipe Properti', 'Sertifikat', 'Daya Listrik']
labels2 = ['Kamar Pembantu', 'Dapur', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Pemandangan', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Tipe Iklan']

# create dictionaries to store the labels and their values
data1 = {label: [np.nan]*1467 for label in labels1}
data2 = {label: [np.nan]*1467 for label in labels2}

loop = 501
print("Progresss: ")

# traverse the hyperlink to get the data needed (test 10 data first)
for i in df['hyperlink'][501:1000]:
    url=i
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # generate random number between 1 and 3
    rand = np.random.randint(1, 17)

    # short pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 5) == 0) and (loop != 0):
        pause = 25 - rand
        time.sleep(pause)
        # print(f'Long Pause for {pause} seconds...')
    
    # Long pause if loop is a multiple of 19, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 19) == 0) and (loop != 0):
        pause = 79 - rand
        print(f'Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    # Super long pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 98) == 0) and (loop != 0):
        pause = 820 - rand
        print(f'Super Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    
    get_price_iklan = soup.find_all('div', class_="r123-listing-summary__price")
    # get the span inside the div
    for i in get_price_iklan:
        get_price_iklan = i.find('span')  # use find instead of find_all
        harga[loop] = get_price_iklan.text

    judul_elements = soup.find_all('h1', {"class":"r123-listing-summary__header-container-title"})
    if judul_elements:
        judul[loop] = judul_elements[0].get_text()

    lokasi_elements = soup.find_all('div', {"class":"r123-listing-summary__header-container-address"})
    if lokasi_elements:
        lokasi[loop] = lokasi_elements[0].get_text()
    
    # ----------------- #

    # first loop
    spec_items = soup.find_all('div', class_="listing-specification-v2__item")
    for item in spec_items:
        label = item.find('span', class_="listing-specification-v2__item-label").text
        value = item.find('span', class_="listing-specification-v2__item-value").text

        # if the label is in the first dictionary, insert the value at the index of the current loop
        if label in data1:
            data1[label][loop] = value

    # second loop
    interior_items = soup.find_all('div', class_="ui-molecule-accordion-r123")
    for item in interior_items:
        spec_items = item.find_all('div', class_="listing-specification-v2__item")
        for spec_item in spec_items:
            label = spec_item.find('span', class_="listing-specification-v2__item-label").text
            value = spec_item.find('span', class_="listing-specification-v2__item-value").text

            # if the label is in the second dictionary, insert the value at the index of the current loop
            if label in data2:
                data2[label][loop] = value
   
    #----------------- #
    loop += 1
    
    # print loop progress total number of link divided by hyperlinks
    print(f'Done: {loop} / {df["hyperlink"].size}')    

Progresss: 
Done: 502 / 1467
Done: 503 / 1467
Done: 504 / 1467
Done: 505 / 1467
Done: 506 / 1467
Done: 507 / 1467
Done: 508 / 1467
Done: 509 / 1467
Done: 510 / 1467
Done: 511 / 1467
Done: 512 / 1467
Done: 513 / 1467
Long Pause for 64 seconds...
Done: 514 / 1467
Done: 515 / 1467
Done: 516 / 1467
Done: 517 / 1467
Done: 518 / 1467
Done: 519 / 1467
Done: 520 / 1467
Done: 521 / 1467
Done: 522 / 1467
Done: 523 / 1467
Done: 524 / 1467
Done: 525 / 1467
Done: 526 / 1467
Done: 527 / 1467
Done: 528 / 1467
Done: 529 / 1467
Done: 530 / 1467
Done: 531 / 1467
Done: 532 / 1467
Long Pause for 63 seconds...
Done: 533 / 1467
Done: 534 / 1467
Done: 535 / 1467
Done: 536 / 1467
Done: 537 / 1467
Done: 538 / 1467
Done: 539 / 1467
Done: 540 / 1467
Done: 541 / 1467
Done: 542 / 1467
Done: 543 / 1467
Done: 544 / 1467
Done: 545 / 1467
Done: 546 / 1467
Done: 547 / 1467
Done: 548 / 1467
Done: 549 / 1467
Done: 550 / 1467
Done: 551 / 1467
Long Pause for 65 seconds...
Done: 552 / 1467
Done: 553 / 1467
Done: 554 / 1467


In [7]:
# get the intersection of the keys in data1 and data2
# duplicate_labels = set(data1.keys()) & set(data2.keys())

# # for each duplicate label, remove it from data2
# for label in duplicate_labels:
#     del data2[label]

# convert the dictionaries to dataframes
judull = pd.DataFrame(judul, columns=['Judul'])
hargaa = pd.DataFrame(harga, columns=['Harga'])
lokasia = pd.DataFrame(lokasi, columns=['Lokasi'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# concatenate the dataframes
df_fix = pd.concat([judull, hargaa, lokasia, df1, df2], axis=1)
print(df_fix.shape)

# drop all rows Judul NaN
df_fix = df_fix.dropna(subset=['Judul'])

df_fix

(1467, 24)


,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
501,RUMAH LUAS MEWAH DENGAN KOLAM RENANG DALAM KAWASAN RUMAH ELITE BALIKPAPAN,"Rp 9,5 Miliar","Balikpapan Selatan, Balikpapan",4,3,670 m²,400 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,13200 Watt,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
502,Rumah Jalan Siaga Balikpapan.,"Rp 2,8 Miliar","Balikpapan Kota, Balikpapan",NaN,NaN,320 m²,110 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3300 Watt,NaN,1,Tidak,Tidak,NaN,1,NaN,Tidak,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
503,Rumah ellite full furnish di tengah kota,"Rp 1,8 Miliar","Balikpapan Utara, Balikpapan",4,4,186 m²,160 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,Bagus,Dijual
504,RUMAH BDI,"Rp 1,7 Miliar","Balikpapan Selatan, Balikpapan",4,3,180 m²,100 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
505,RUMAH LUAS MEWAH DEPAN TAMAN DAN LAPANGAN BASKET DALAM PERUMAHAN ELITE BALIKPAPAN,"Rp 9,5 Miliar","Balikpapan Baru, Balikpapan",4,4,396 m²,289 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,RUMAH + TANAH LUAS PINGGIR JALAN RAYA,Rp 12 Miliar,"Balikpapan Utara, Balikpapan",6,5,1270 m²,500 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
996,RUMAH PRAPATAN,Rp 430 Juta,"Prapatan, Balikpapan",3,1,180 m²,60 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
997,RUMAH MINIMALIS WIKA MAHOGANY,Rp 700 Juta,"Balikpapan Utara, Balikpapan",2,2,120 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
998,RUMAH MINIMALIS DIKAWASAN WIKA,Rp 750 Juta,"Balikpapan Utara, Balikpapan",2,2,125 m²,70 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual


In [8]:
# to csv
df_fix.to_csv('./datasets/rumah123_data_single_raw_fix_part_2.csv', index=True)

In [9]:
# full inner join on judul and Judul delete the rest that not match
df_fix_combine = pd.merge(df, df_fix, left_on='judul', right_on='Judul', how='inner')

# drop where Judul is NaN
df_fix_combine = df_fix_combine.dropna(subset=['Judul'])

In [10]:
df_fix_combine.sample(10)

,judul,hyperlink,img_hyperlinks,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
294,RUMAH ETNIK DI BATU AMPAR DENGAN KONSEP CONTTAGE ASRI DAN NYAMAN,https://www.rumah123.com//properti/balikpapan/hos13884122/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13884122/original/hos13884122-rumah-di...,RUMAH ETNIK DI BATU AMPAR DENGAN KONSEP CONTTAGE ASRI DAN NYAMAN,"Rp 4,25 Miliar","Batu Ampar, Balikpapan",5,6,770 m²,350 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
266,rumah model Belanda bisa tempat tinggal ada usaha kost,https://www.rumah123.com//properti/balikpapan/hos14630080/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14630080/original/hos14630080-rumah-di...,rumah model Belanda bisa tempat tinggal ada usaha kost,Rp 970 Juta,"Balikpapan Selatan, Balikpapan",5,3,120 m²,185 m²,2,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,NaN,Pemukiman Warga,Tidak,3 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
63,TAMAN SARI BUKIT MUTIARA (WIKA),https://www.rumah123.com//properti/balikpapan/hos9907448/,https://picture.rumah123.com/r123/720x420-crop/house/ho99/9907448/original/hos9907448-rumah-di-j...,TAMAN SARI BUKIT MUTIARA (WIKA),"Rp 1,5 Miliar","Balikpapan Utara, Balikpapan",4,2,200 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
257,RUMAH MINIMALIS 2 LANTAI SIAP HUNI DENGAN TAMBAHAN TANAH YANG CUKUP LUAS,https://www.rumah123.com//properti/balikpapan/hos14836211/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14836211/original/hos14836211-rumah-di...,RUMAH MINIMALIS 2 LANTAI SIAP HUNI DENGAN TAMBAHAN TANAH YANG CUKUP LUAS,"Rp 1,25 Miliar","Balikpapan Utara, Balikpapan",2,2,60 m²,47 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
68,BALIKPAPAN BARU,https://www.rumah123.com//properti/balikpapan/hos9907443/,https://picture.rumah123.com/r123/720x420-crop/house/ho99/9907443/original/hos9907443-rumah-di-j...,BALIKPAPAN BARU,"Rp 2,5 Miliar","Damai, Balikpapan",4,2,200 m²,275 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
45,BALIKPAPAN PERUMAHAN REGENCY,https://www.rumah123.com//properti/balikpapan/hos14863499/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14863499/original/hos14863499-rumah-di...,BALIKPAPAN PERUMAHAN REGENCY,"Rp 1,75 Miliar","Balikpapan Selatan, Balikpapan",3,2,152 m²,115 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
155,Hunian modern dekat rumah sakit umum Kanujoso,https://www.rumah123.com//properti/balikpapan/hos16150027/,https://picture.rumah123.com/r123/720x420-crop/house/ho16/16150027/original/hos16150027-rumah-di...,Hunian modern dekat rumah sakit umum Kanujoso,"Rp 1,01 Miliar","Balikpapan Utara, Balikpapan",4,2,120 m²,72 m²,2,Rumah,HGB - Hak Guna Bangunan,2200 Watt,NaN,NaN,Ya,Ya,NaN,NaN,Pemukiman Warga,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
210,HARGA SPESIAL RUMAH KANTOR 2LT DI TENGAH KOTA,https://www.rumah123.com//properti/balikpapan/hos15688469/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15688469/original/hos15688469-rumah-di...,HARGA SPESIAL RUMAH KANTOR 2LT DI TENGAH KOTA,"Rp 2,1 Miliar","Balikpapan Tengah, Balikpapan",8,6,429 m²,429 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
376,Bu Rumah Komplek Full Furnish di Jl Wonorejo Balikpapan Utara,https://www.rumah123.com//properti/balikpapan/hos16140392/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1476447/2024-01-23-07-02-24-03562...,Bu Rumah Komplek Full Furnish di Jl Wonorejo Balikp

In [11]:
# check null values
df_fix_combine.isnull().sum()

judul                    0
hyperlink                0
img_hyperlinks           0
Judul                    0
Harga                    0
Lokasi                   0
Kamar Tidur             15
Kamar Mandi             12
Luas Tanah               0
Luas Bangunan            0
Carport                546
Tipe Properti            0
Sertifikat               0
Daya Listrik            51
Kamar Pembantu         533
Dapur                  533
Ruang Makan            330
Ruang Tamu             330
Kondisi Perabotan      477
Jumlah Lantai          458
Pemandangan            533
Terjangkau Internet    330
Lebar Jalan            534
Sumber Air             527
Hook                   330
Kondisi Properti       435
Tipe Iklan               0
dtype: int64

In [12]:
# to csv
df_fix_combine.to_csv('./datasets/rumah123_fix_data_fixxx_part_2.csv')